In [1]:
import pandas as pd
from konlpy.tag import Twitter

In [2]:
# 사진 제외 : 87%
# 사진 포함 : 88%

In [3]:
df_naver = pd.read_csv('A_V_Adj_C_df_naver_post_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep = pd.read_csv('A_V_Adj_C_df_dbdbdeep_post_table.csv',sep='\t',encoding='utf-8')
df_naver_page = pd.read_csv('A_V_Adj_C_df_naver_page_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep_page = pd.read_csv('A_V_Adj_C_df_dbdbdeep_page_table.csv',sep='\t',encoding='utf-8')

In [4]:
Total = pd.concat([df_naver,df_dbdbdeep])

In [5]:
Origin_Total = Total

In [6]:
Total = Total.dropna(subset=['Structure_tag'])

In [7]:
Total['only_tag'] = Total['Structure_tag'].apply(lambda x : " img ".join(list(map(lambda x : 'text' if len(x)>3 else '', x.split('<img>')))).strip().replace('  ',' '))

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## Doc2vec

In [8]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
Total = Total[['only_tag','label']]

In [10]:
text = Total['only_tag'].tolist()

In [11]:
y = Total['label'].tolist()

In [12]:
sentences = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(text)]

In [13]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [14]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [15]:
doc_vectorizer = Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=5,        # distance between the predicted word and context words
    size=13,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=20,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=cores,   # multi cpu
    hs = 1,          # hierarchical softmax / default 0
    negative = 10,   # negative sampling / default 5
)

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2018-09-06 17:37:18,331 : WARNING : consider setting layer size to a multiple of 4 for greater performance


In [16]:
doc_vectorizer.build_vocab(sentences)

2018-09-06 17:37:18,337 : INFO : collecting all words and their counts
2018-09-06 17:37:18,338 : WARNING : Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2018-09-06 17:37:18,340 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-09-06 17:37:18,448 : INFO : PROGRESS: at example #10000, processed 1110363 words (10350251/s), 7 word types, 10000 tags
2018-09-06 17:37:18,507 : INFO : collected 7 word types and 16260 unique tags from a corpus of 16260 examples and 1702812 words
2018-09-06 17:37:18,508 : INFO : Loading a fresh vocabulary
2018-09-06 17:37:18,508 : INFO : min_count=20 retains 7 unique words (100% of original 7, drops 0)
2018-09-06 17:37:18,509 : INFO : min_count=20 leaves 1702812 word corpus (100% of original 1702812, drops 0)
2018-09-06 17:37:18,509 : INFO : deleting the raw counts dictionary of 7 items
2018-09-06 17:37:18,510 : INFO : sample=0.001 downsamples 7 most-common 

In [17]:
print(str(doc_vectorizer))

Doc2Vec(dbow+w,d13,n10,hs,w5,mc20,s0.001,t12)


In [18]:
doc_vectorizer.corpus_count*0.03

487.79999999999995

In [19]:
doc_vectorizer.iter

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


5

In [20]:
import time
start = time.time()
for epoch in range(10):
    doc_vectorizer.train(sentences, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
end = time.time()
print("During Time: {}".format(end-start))

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.
2018-09-06 17:37:18,748 : INFO : training model with 12 workers on 7 vocabulary and 13 features, using sg=1 hs=1 sample=0.001 negative=10 window=5
2018-09-06 17:37:19,540 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-06 17:37:19,540 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-06 17:37:19,551 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-06 17:37:19,554 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-06 17:37:19,570 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-06 17:37:19,572 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-06 17:37:19,573 : INFO : worker thread finished; awaiting finish of 5 

2018-09-06 17:37:24,551 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-06 17:37:24,557 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-06 17:37:24,562 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-06 17:37:24,578 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-06 17:37:24,578 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-06 17:37:24,585 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-06 17:37:24,588 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-06 17:37:24,589 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-06 17:37:24,589 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-06 17:37:24,590 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-06 17:37:24,590 : INFO : EPOCH - 2 : training on 1702812 raw words (168024 effecti

2018-09-06 17:37:29,624 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-06 17:37:29,624 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-06 17:37:29,625 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-06 17:37:29,625 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-06 17:37:29,626 : INFO : EPOCH - 3 : training on 1702812 raw words (169128 effective words) took 0.8s, 203694 effective words/s
2018-09-06 17:37:30,422 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-06 17:37:30,423 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-06 17:37:30,427 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-06 17:37:30,430 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-06 17:37:30,438 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-06 17:37:30,447 : INFO : worker thr

2018-09-06 17:37:35,421 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-06 17:37:35,426 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-06 17:37:35,442 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-06 17:37:35,443 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-06 17:37:35,444 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-06 17:37:35,448 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-06 17:37:35,459 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-06 17:37:35,460 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-06 17:37:35,461 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-06 17:37:35,462 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-06 17:37:35,462 : INFO : worker thread finished; awaiting finish of 0 more thread

2018-09-06 17:37:40,601 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-06 17:37:40,604 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-06 17:37:40,606 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-06 17:37:40,615 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-06 17:37:40,617 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-06 17:37:40,619 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-06 17:37:40,620 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-06 17:37:40,620 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-06 17:37:40,620 : INFO : EPOCH - 1 : training on 1702812 raw words (169333 effective words) took 0.9s, 197310 effective words/s
2018-09-06 17:37:41,415 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-06 17:37:41,418 : INFO : worker thre

2018-09-06 17:37:45,611 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-06 17:37:45,612 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-06 17:37:45,612 : INFO : EPOCH - 2 : training on 1702812 raw words (168933 effective words) took 0.8s, 208346 effective words/s
2018-09-06 17:37:46,391 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-06 17:37:46,392 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-06 17:37:46,402 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-06 17:37:46,403 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-06 17:37:46,404 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-06 17:37:46,413 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-06 17:37:46,415 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-06 17:37:46,428 : INFO : worker thr

2018-09-06 17:37:51,313 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-09-06 17:37:51,322 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-09-06 17:37:51,334 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-09-06 17:37:51,338 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-09-06 17:37:51,345 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-09-06 17:37:51,346 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-06 17:37:51,346 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-06 17:37:51,347 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-06 17:37:51,347 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-06 17:37:51,348 : INFO : EPOCH - 4 : training on 1702812 raw words (169079 effective words) took 0.8s, 210008 effective words/s
2018-09-06 17:37:52,118 : INFO : worker threa

2018-09-06 17:37:56,268 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-06 17:37:56,269 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-06 17:37:56,269 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-06 17:37:56,269 : INFO : EPOCH - 5 : training on 1702812 raw words (168262 effective words) took 0.8s, 209641 effective words/s
2018-09-06 17:37:56,270 : INFO : training on a 8514060 raw words (845085 effective words) took 4.1s, 205744 effective words/s
2018-09-06 17:37:56,270 : INFO : training model with 12 workers on 7 vocabulary and 13 features, using sg=1 hs=1 sample=0.001 negative=10 window=5
2018-09-06 17:37:57,048 : INFO : worker thread finished; awaiting finish of 11 more threads
2018-09-06 17:37:57,049 : INFO : worker thread finished; awaiting finish of 10 more threads
2018-09-06 17:37:57,053 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-09-06 17:37:57,061 : INFO : worker thr

During Time: 41.63170409202576


## Model_save

In [21]:
model_name = '[Structure_tag]Doc2vec_features_100.pkl.model'
doc_vectorizer.save(model_name)

2018-09-06 17:38:00,384 : INFO : saving Doc2Vec object under [Structure_tag]Doc2vec_features_100.pkl.model, separately None
2018-09-06 17:38:00,425 : INFO : saved [Structure_tag]Doc2vec_features_100.pkl.model


## model_load

In [22]:
doc_vectorizer = Doc2Vec.load(model_name)

2018-09-06 17:38:00,443 : INFO : loading Doc2Vec object from [Structure_tag]Doc2vec_features_100.pkl.model
2018-09-06 17:38:00,466 : INFO : loading vocabulary recursively from [Structure_tag]Doc2vec_features_100.pkl.model.vocabulary.* with mmap=None
2018-09-06 17:38:00,467 : INFO : loading wv recursively from [Structure_tag]Doc2vec_features_100.pkl.model.wv.* with mmap=None
2018-09-06 17:38:00,469 : INFO : loading trainables recursively from [Structure_tag]Doc2vec_features_100.pkl.model.trainables.* with mmap=None
2018-09-06 17:38:00,470 : INFO : loading docvecs recursively from [Structure_tag]Doc2vec_features_100.pkl.model.docvecs.* with mmap=None
2018-09-06 17:38:00,470 : INFO : loaded [Structure_tag]Doc2vec_features_100.pkl.model


In [23]:
sentences

[TaggedDocument(words='text img text img', tags=['0']),
 TaggedDocument(words='text img text img', tags=['1']),
 TaggedDocument(words='text img text img text', tags=['2']),
 TaggedDocument(words='text img text img text', tags=['3']),
 TaggedDocument(words='text img text img', tags=['4']),
 TaggedDocument(words='text img text img', tags=['5']),
 TaggedDocument(words='text img text img text', tags=['6']),
 TaggedDocument(words='text img text img', tags=['7']),
 TaggedDocument(words='text img text img text', tags=['8']),
 TaggedDocument(words='text img text img text', tags=['9']),
 TaggedDocument(words='img text img text img', tags=['10']),
 TaggedDocument(words='img text img text img text', tags=['11']),
 TaggedDocument(words='img text img text img text img text img text img text', tags=['12']),
 TaggedDocument(words='img text img text img text img text img text img text img text img text', tags=['13']),
 TaggedDocument(words='img text img text img text', tags=['14']),
 TaggedDocument(wo

In [24]:
x = Total['only_tag'].tolist()

In [25]:
doc2vec_sentences = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(x)]

In [26]:
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in doc2vec_sentences]

In [27]:
Text_df = pd.DataFrame({'word':X_train,'label':y})

In [28]:
from sklearn.utils import shuffle
Text_df = shuffle(Text_df)

In [29]:
Text_df = Text_df.dropna()

In [30]:
X_train = Text_df['word'].tolist()
y = Text_df['label'].tolist()

## Data_split

In [31]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train , y , test_size=0.33, random_state=42)

## SVM

In [32]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.6278419679463287


In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result.predict(x_test)))

             precision    recall  f1-score   support

          0       0.55      0.13      0.21      2045
          1       0.64      0.93      0.76      3321

avg / total       0.60      0.63      0.55      5366



In [34]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(result, X_train, y, cv=5)
scores                                              

array([0.61297264, 0.61439114, 0.61715867, 0.61439114, 0.60965857])

## Gausian NB

In [35]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x_train, y_train)
clf.predict(x_test)
print("NB :",accuracy_score(clf.predict(x_test),y_test))

NB : 0.5447260529258293


In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(x_test)))

             precision    recall  f1-score   support

          0       0.43      0.64      0.52      2045
          1       0.69      0.49      0.57      3321

avg / total       0.59      0.54      0.55      5366



In [37]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_train, y, cv=5)
scores                                              

array([0.54595758, 0.53905289, 0.53474785, 0.54458795, 0.54567825])

## Randomforest

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(x_train, y_train)

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [39]:
print("RF :",accuracy_score(clf.predict(x_test),y_test))

RF : 0.6188967573611629


In [40]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(x_test)))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      2045
          1       0.62      1.00      0.76      3321

avg / total       0.38      0.62      0.47      5366



C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [41]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_train, y, cv=5)
scores                                              

array([0.60897633, 0.60885609, 0.60885609, 0.60885609, 0.60904337])

## Logsitic Regression

In [42]:
from  sklearn  import  linear_model
logreg = linear_model.LogisticRegression(C=2.0,random_state=42,solver='sag',multi_class='multinomial',warm_start=True)
logreg.fit(x_train, y_train)

LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='sag',
          tol=0.0001, verbose=0, warm_start=True)

In [43]:
print("RF :",accuracy_score(logreg.predict(x_test),y_test))

RF : 0.6250465896384644


In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test, logreg.predict(x_test)))

             precision    recall  f1-score   support

          0       0.53      0.14      0.22      2045
          1       0.64      0.93      0.75      3321

avg / total       0.60      0.63      0.55      5366



In [45]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(logreg, X_train, y, cv=5)
scores                                              

array([0.61389487, 0.61408364, 0.61562116, 0.61654367, 0.60658259])

## Decision Tree 

In [46]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)

In [47]:
print("DT :",accuracy_score(clf.predict(x_test),y_test))


DT : 0.575475214312337


In [48]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(x_test)))

             precision    recall  f1-score   support

          0       0.45      0.47      0.46      2045
          1       0.66      0.64      0.65      3321

avg / total       0.58      0.58      0.58      5366



In [49]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_train, y, cv=5)
scores                                              

array([0.59637258, 0.59163592, 0.58671587, 0.58425584, 0.579514  ])

## MLP

In [50]:
import time
from sklearn.neural_network import MLPClassifier

In [51]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(50,),
    max_iter=35,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=.1
)

In [52]:
start = time.time()
mlp_clf.fit(x_train, y_train)
end = time.time()
print('Time: {:f}s'.format(end-start))

Iteration 1, loss = 0.66780073
Iteration 2, loss = 0.66358300
Iteration 3, loss = 0.66131164
Iteration 4, loss = 0.65846484
Iteration 5, loss = 0.65523987
Iteration 6, loss = 0.65142760
Iteration 7, loss = 0.64845503
Iteration 8, loss = 0.64435392
Iteration 9, loss = 0.64150340
Iteration 10, loss = 0.63965882
Iteration 11, loss = 0.63689826
Iteration 12, loss = 0.63434949
Iteration 13, loss = 0.63230845
Iteration 14, loss = 0.63040799
Iteration 15, loss = 0.62928761
Iteration 16, loss = 0.62783311
Iteration 17, loss = 0.62589285
Iteration 18, loss = 0.62488952
Iteration 19, loss = 0.62409348
Iteration 20, loss = 0.62278827
Iteration 21, loss = 0.62181048
Iteration 22, loss = 0.62155187
Iteration 23, loss = 0.62198576
Iteration 24, loss = 0.62066342
Iteration 25, loss = 0.61941992
Iteration 26, loss = 0.61946351
Iteration 27, loss = 0.61808495
Iteration 28, loss = 0.61788015
Iteration 29, loss = 0.61642344
Iteration 30, loss = 0.61639642
Iteration 31, loss = 0.61595743
Iteration 32, los

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (35) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [53]:
y_pred = mlp_clf.predict(x_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))

테스트 정확도: 0.650


In [54]:
from sklearn.metrics import classification_report
print(classification_report(y_test, mlp_clf.predict(x_test)))

             precision    recall  f1-score   support

          0       0.56      0.38      0.45      2045
          1       0.68      0.82      0.74      3321

avg / total       0.64      0.65      0.63      5366



In [55]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(mlp_clf, X_train, y, cv=5)
scores                                              

Iteration 1, loss = 0.66472102
Iteration 2, loss = 0.66106546
Iteration 3, loss = 0.65804422
Iteration 4, loss = 0.65513551
Iteration 5, loss = 0.65346526
Iteration 6, loss = 0.64727140
Iteration 7, loss = 0.64380635
Iteration 8, loss = 0.64098094
Iteration 9, loss = 0.63957165
Iteration 10, loss = 0.63604730
Iteration 11, loss = 0.63301276
Iteration 12, loss = 0.63002191
Iteration 13, loss = 0.62851712
Iteration 14, loss = 0.62709170
Iteration 15, loss = 0.62508055
Iteration 16, loss = 0.62574230
Iteration 17, loss = 0.62456145
Iteration 18, loss = 0.62337445
Iteration 19, loss = 0.62176159
Iteration 20, loss = 0.62660635
Iteration 21, loss = 0.62219902
Iteration 22, loss = 0.61938186
Iteration 23, loss = 0.61979277
Iteration 24, loss = 0.62052941
Iteration 25, loss = 0.61737960
Iteration 26, loss = 0.61664722
Iteration 27, loss = 0.61604414
Iteration 28, loss = 0.62310692
Iteration 29, loss = 0.61714128
Iteration 30, loss = 0.61549058
Iteration 31, loss = 0.61408513
Iteration 32, los

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (35) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 8, loss = 0.63800875
Iteration 9, loss = 0.63499069
Iteration 10, loss = 0.63201602
Iteration 11, loss = 0.63128645
Iteration 12, loss = 0.62799130
Iteration 13, loss = 0.62754679
Iteration 14, loss = 0.62420763
Iteration 15, loss = 0.62391119
Iteration 16, loss = 0.62308049
Iteration 17, loss = 0.62246158
Iteration 18, loss = 0.61947233
Iteration 19, loss = 0.61946986
Iteration 20, loss = 0.61947517
Iteration 21, loss = 0.61917459
Iteration 22, loss = 0.61875947
Iteration 23, loss = 0.61949274
Iteration 24, loss = 0.61638037
Iteration 25, loss = 0.61564036
Iteration 26, loss = 0.61766314
Iteration 27, loss = 0.61580249
Iteration 28, loss = 0.61508072
Iteration 29, loss = 0.61733890
Iteration 30, loss = 0.61533186
Iteration 31, loss = 0.61381795
Iteration 32, loss = 0.61660256
Iteration 33, loss = 0.61215926
Iteration 34, loss = 0.61873671
Iteration 35, loss = 0.61323126


C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (35) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


array([0.65600984, 0.63345633, 0.63437884, 0.61900369, 0.63580437])